In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# Carga las stop-words en español
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Sube hasta la carpeta padre (de scripts/ a la raíz de tu proyecto)
os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
print("Directorio de trabajo actual:", os.getcwd())


Directorio de trabajo actual: c:\Users\Pablo\OneDrive - Universidad Loyola Andalucía\Master\TFM\cv-matching


In [6]:
# 1. Configuración de rutas ───────────────────────────────────────────
# Ajusta estos paths según tu estructura de carpetas:
train_cvs_path = r'data\interim\cvs_texto_train.csv'
test_cvs_path  = r'data/interim/cvs_texto_test.csv'

# Lista de ficheros de ofertas y la categoría que representan:
offer_files = {
    'Ciencia_de_datos'       : r'data/interim/Ciencia_de_datos_España_ofertas.csv',
    'Ingeniero_de_datos'     : r'data/interim/Ingeniero_de_datos_España_ofertas.csv',
    'Jurista'                : r'data/interim/Jurista_Málaga_ofertas.csv',
    'Traductor_de_inglés'    : r'data/interim/Traductor_de_inglés_Málaga_ofertas.csv',
}

# Mapeo de código en nombre de CV → categoría
mapping = {
    'cdatos'    : 'Ciencia_de_datos',
    'ingdatos'  : 'Ingeniero_de_datos',
    'jurista'   : 'Jurista',
    'traductor' : 'Traductor_de_inglés'
}

In [9]:
# 2. Carga de los CVs de entrenamiento ────────────────────────────────
df_cvs = pd.read_csv(train_cvs_path)
# Extraer el "código" (tercera parte al split por '_') y mapear:
df_cvs['code'] = df_cvs['Nombre del archivo'].str.split('_').str[2]
df_cvs['category'] = df_cvs['code'].map(mapping)
df_cvs = df_cvs.rename(columns={'Texto extraído':'cv_text'})
df_cvs = df_cvs[['Nombre del archivo','cv_text','category']].rename(columns={'Nombre del archivo':'cv_id'})

In [27]:
# 3. Carga de las ofertas y etiquetado ─────────────────────────────────
offers_list = []
for cat, path in offer_files.items():
    if not os.path.exists(path):
        raise FileNotFoundError(f"No existe el fichero de ofertas: {path}")
    tmp = pd.read_csv(path)
    tmp = tmp.rename(columns={'descripcion_oferta':'offer_text'})
    tmp['offer_id']  = tmp.index.astype(str) + f'_{cat}'
    tmp['category']  = cat
    offers_list.append(tmp[['offer_id','offer_text','category']])
offers = pd.concat(offers_list, ignore_index=True)

offers = offers.rename(columns={'category':'offer_category'})

In [11]:
# 4. Construcción del dataset de pares CV–oferta ───────────────────────
# Cross join (producto cartesiano) y etiqueta binaria:
df_cvs['key']   = 1
offers['key']   = 1
pairs = df_cvs.merge(offers, on='key').drop('key', axis=1)

pairs['label']  = (pairs['category_x'] == pairs['category_y']).astype(int)
pairs = pairs.rename(columns={'category_x':'cv_category',
                              'category_y':'offer_category'})

In [18]:
# 5. TF–IDF + SVM en Pipeline con validación cruzada ───────────────────

X = pairs['cv_text'] + ' ' + pairs['offer_text']
y = pairs['label']

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf',   LinearSVC(class_weight='balanced', random_state=42))
])

# Definimos un grid de ejemplo
param_grid = {
    # TF–IDF
    'tfidf__stop_words':    [spanish_stopwords],
    'tfidf__ngram_range':   [(1,1), (1,2)],
    'tfidf__sublinear_tf':  [True, False],
    'tfidf__min_df':        [1, 2, 3],
    'tfidf__max_df':        [0.5, 0.75, 1.0],
    # SVM
    'clf__C':               [0.01, 0.1, 1, 10],
    'clf__loss':            ['hinge', 'squared_hinge'],
    'clf__max_iter':        [1000, 5000]
}

search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',
    cv=cv,            # aquí tu StratifiedKFold de antes
    n_jobs=-1,        # usa todos los cores disponibles
    verbose=2,
    refit=True        # al acabar, deja el pipeline con los mejores params
)

# Lanza la búsqueda
search.fit(X, y)

# Resultados
print("Mejor F1 (CV):", search.best_score_)
print("Mejores parámetros:")
for param, val in search.best_params_.items():
    print(f"  - {param}: {val}")

# Si quieres inspeccionar varios de los mejores resultados:
results = pd.DataFrame(search.cv_results_)[
    ['params','mean_test_score','std_test_score']
].sort_values('mean_test_score', ascending=False).head(10)
print(results)


Fitting 5 folds for each of 576 candidates, totalling 2880 fits
Mejor F1 (CV): 0.6064759211840609
Mejores parámetros:
  - clf__C: 10
  - clf__loss: squared_hinge
  - clf__max_iter: 1000
  - tfidf__max_df: 1.0
  - tfidf__min_df: 1
  - tfidf__ngram_range: (1, 1)
  - tfidf__stop_words: ['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros'

In [21]:
# Mostramos la lista de los parametros de aquellos modelos que estan por encima de 0.6
print("Modelos con F1 > 0.6:")

# Mostramos la lista de 'params'
for i in range(len(results)):
    if results.iloc[i]['mean_test_score'] > 0.6:
        print(results.iloc[i]['params'])

Modelos con F1 > 0.6:
{'clf__C': 10, 'clf__loss': 'squared_hinge', 'clf__max_iter': 1000, 'tfidf__max_df': 1.0, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 1), 'tfidf__stop_words': ['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', '

In [ ]:
# 1) Fija en el Pipeline los parámetros "comunes" a todos los top-6:
pipeline_refined = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=spanish_stopwords,  # igual en todos
        ngram_range=(1,1),             # unigramas, igual en todos
        sublinear_tf=True,             # igual en todos
        max_df=1.0                     # igual en todos
    )),
    ('clf', LinearSVC(
        C=10,                          # igual en todos
        loss='squared_hinge',         # igual en todos
        class_weight='balanced',
        random_state=42
    ))
])

# 2) Solo param_grid para lo que varió:
param_grid_refined = {
    'tfidf__min_df':    [1, 2, 3, 4, 5],            # antes 1–3, añadimos 4 y 5
    'clf__max_iter':    [1000, 2000, 5000, 10000],  # antes 1000 y 5000, añadimos 2000 y 10000
}

search_refined = GridSearchCV(
    estimator=pipeline_refined,
    param_grid=param_grid_refined,
    scoring='f1',
    cv=cv,        # tu StratifiedKFold
    n_jobs=-1,
    verbose=2,
    refit=True
)

# 3) Lanza la búsqueda:
search_refined.fit(X, y)

# 4) Resultado:
print("Mejor F1 (CV):", search_refined.best_score_)
print("Mejores parámetros:")
for p, v in search_refined.best_params_.items():
    print(f"  - {p}: {v}")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Mejor F1 (CV): 0.6064759211840609
Mejores parámetros:
  - clf__max_iter: 1000
  - tfidf__min_df: 1


In [32]:
from sklearn.svm import SVC

pipeline_proba = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf',   SVC(class_weight='balanced', probability=True, random_state=42))
])

# (Repite aquí tu GridSearchCV, adaptando los nombres de los parámetros si quieres
#  buscar 'clf__C', o 'clf__kernel', etc. — según lo que te interese.)

search_proba = GridSearchCV(
    pipeline_proba,
    param_grid_refined,
    scoring='f1',
    cv=cv,
    n_jobs=-1,
    verbose=2,
    refit=True
)
search_proba.fit(X, y)
print("Mejor F1 (CV):", search_proba.best_score_)
print("Mejores parámetros:")
for p, v in search_proba.best_params_.items():
    print(f"  - {p}: {v}")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Mejor F1 (CV): 0.6913560287288757
Mejores parámetros:
  - clf__max_iter: 2000
  - tfidf__min_df: 1


c:\Users\Pablo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [36]:
# ─── 6. Guardar el mejor modelo ────────────────────────────────────────
best_model = search_proba.best_estimator_
print("Parámetros del mejor modelo:", search_refined.best_params_)

Parámetros del mejor modelo: {'clf__max_iter': 1000, 'tfidf__min_df': 1}


In [41]:
# ─── 7. Ranking de ofertas para cada CV de test ────────────────────────
# (asumiendo que ya tienes `offers` cargado como antes)

# 7.1. Carga los CVs de test
df_test = pd.read_csv(test_cvs_path)
df_test['code']     = df_test['Nombre del archivo'].str.split('_').str[2]
df_test['category'] = df_test['code'].map(mapping)
df_test = df_test.rename(columns={'Texto extraído':'cv_text',
                                  'Nombre del archivo':'cv_id'})
df_test = df_test[['cv_id','cv_text','category']]

# 7.2. Para cada CV, calcula decision_function sobre todas las ofertas
rankings = []
for _, row in df_test.iterrows():
    cv_id   = row['cv_id']
    text_cv = row['cv_text']

    # construye batch de pares CV+oferta
    batch = [text_cv + ' ' + off for off in offers['offer_text'].tolist()]
    scores_raw  = best_model.decision_function(batch)
    scores_norm = 1 / (1 + np.exp(-scores_raw))   # sigmoide: values ∈ (0,1)

    df_rank = pd.DataFrame({
        'cv_id'         : cv_id,
        'offer_id'      : offers['offer_id'],
        'offer_category': offers['offer_category'],
        'score'         : scores_norm              # ahora en [0,1]
    })
    df_rank = df_rank.sort_values('score', ascending=False).reset_index(drop=True)
    df_rank['rank'] = df_rank.index + 1

    rankings.append(df_rank)

df_rankings = pd.concat(rankings, ignore_index=True)
df_rankings.to_csv('rankings_test.csv', index=False)
print("Rankings de test guardados en 'rankings_test.csv'")

Rankings de test guardados en 'rankings_test.csv'
